In [1]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

import catboost as cb
from sklearn import model_selection, metrics, linear_model, ensemble, naive_bayes, neighbors, svm, tree
from xgboost import XGBClassifier
import lightgbm

import os
import gc
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv
/kaggle/input/playground-series-s3e3/sample_submission.csv
/kaggle/input/playground-series-s3e3/train.csv
/kaggle/input/playground-series-s3e3/test.csv


In [2]:
train = pd.read_csv(r'/kaggle/input/playground-series-s3e3/train.csv')
train_o = pd.read_csv(r'/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
test = pd.read_csv(r'/kaggle/input/playground-series-s3e3/test.csv')
ss = pd.read_csv(r'/kaggle/input/playground-series-s3e3/sample_submission.csv')

In [3]:
print(train.shape, train_o.shape, test.shape)

(1677, 35) (1470, 35) (1119, 34)


In [4]:
train.isnull().sum() + train_o.isnull().sum()

Age                         0.0
Attrition                   0.0
BusinessTravel              0.0
DailyRate                   0.0
Department                  0.0
DistanceFromHome            0.0
Education                   0.0
EducationField              0.0
EmployeeCount               0.0
EmployeeNumber              NaN
EnvironmentSatisfaction     0.0
Gender                      0.0
HourlyRate                  0.0
JobInvolvement              0.0
JobLevel                    0.0
JobRole                     0.0
JobSatisfaction             0.0
MaritalStatus               0.0
MonthlyIncome               0.0
MonthlyRate                 0.0
NumCompaniesWorked          0.0
Over18                      0.0
OverTime                    0.0
PercentSalaryHike           0.0
PerformanceRating           0.0
RelationshipSatisfaction    0.0
StandardHours               0.0
StockOptionLevel            0.0
TotalWorkingYears           0.0
TrainingTimesLastYear       0.0
WorkLifeBalance             0.0
YearsAtC

In [5]:
display(train.head())
display(train_o.head())

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,...,80,1,10,2,3,10,0,7,8,0
1,1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,...,80,1,4,3,3,4,2,0,3,0
2,2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,...,80,2,4,3,3,3,2,1,2,0
3,3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,...,80,0,15,1,1,6,0,0,2,0
4,4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,...,80,0,31,0,3,31,14,4,10,1


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [6]:
display(train['Attrition'].value_counts())
display(train_o['Attrition'].value_counts())

0    1477
1     200
Name: Attrition, dtype: int64

No     1233
Yes     237
Name: Attrition, dtype: int64

In [7]:
train_o['Attrition'] = train_o['Attrition'].map(lambda x: 1 if x == "Yes" else 0)
train_o['id'] = train_o['EmployeeNumber']

In [8]:
train_all = pd.concat([train, train_o.drop('EmployeeNumber', axis = 1)])

In [9]:
display(train_all.shape)
display(train_all['Attrition'].value_counts())

(3147, 35)

0    2710
1     437
Name: Attrition, dtype: int64

In [10]:
target = train_all['Attrition']

data = pd.concat([train_all.drop('Attrition', axis = 1), test]).reset_index(drop = True)

print(train_all.shape, test.shape, data.shape)

(3147, 35) (1119, 34) (4266, 34)


In [11]:
text_features = []

for column in data.columns:
    if data[column].dtype == 'object':
        text_features.append(column)

for text_feature in text_features:
    data = data.join(pd.get_dummies(data[text_feature], prefix=text_feature))
    data = data.drop(text_feature, axis = 1)

In [12]:
data.head()

,id,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0,36,599,24,3,1,4,42,3,1,...,0,0,0,0,0,1,0,1,0,1
1,1,35,921,8,3,1,1,46,3,1,...,0,0,0,1,0,1,0,1,1,0
2,2,32,718,26,3,1,3,80,3,2,...,0,0,1,0,1,0,0,1,1,0
3,3,38,1488,2,3,1,3,40,3,2,...,0,0,0,0,0,1,0,1,1,0
4,4,50,1017,5,4,1,2,37,3,5,...,0,0,0,0,0,0,1,1,0,1


In [13]:
df_train = data.iloc[:len(target), :]

df_test = data.iloc[len(target):, :]

df_train.shape, target.shape, df_test.shape

((3147, 55), (3147,), (1119, 55))

In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_train.drop('id', axis = 1), target, random_state = 42)

CATBOOST

In [15]:
model_cb = cb.CatBoostClassifier(depth = 3,
                                 learning_rate = 0.01,
                                 rsm = 0.5,
                                 random_seed = 42,
                                 verbose = False).fit(X_train, y_train)
print(metrics.roc_auc_score(model_cb.predict(X_test), y_test))
preds_cb = model_cb.predict(df_test.drop('id', axis = 1))
proba_cb = model_cb.predict_proba(df_test.drop('id', axis = 1))

0.8764324324324325


XGBOOST

In [16]:
model_xgb = XGBClassifier().fit(X_train, y_train)
print(metrics.roc_auc_score(model_xgb.predict(X_test), y_test))
preds_xgb = model_xgb.predict(df_test.drop('id', axis = 1))
proba_xgb = model_cb.predict_proba(df_test.drop('id', axis = 1))

0.7526585094549498


LIGHTGBM

In [17]:
model_lgbm = lightgbm.LGBMClassifier().fit(X_train, y_train)
print(metrics.roc_auc_score(model_lgbm.predict(X_test), y_test))
preds_lgbm = model_lgbm.predict(df_test.drop('id', axis = 1))
proba_lgbm = model_lgbm.predict_proba(df_test.drop('id', axis = 1))

0.7554751430868927


In [18]:
proba_global = (proba_cb+proba_xgb+proba_lgbm)/3
proba_global

array([[0.73188835, 0.26811165],
       [0.87269235, 0.12730765],
       [0.92160381, 0.07839619],
       ...,
       [0.96059179, 0.03940821],
       [0.96705772, 0.03294228],
       [0.98617029, 0.01382971]])

In [19]:
proba_global[0]

array([0.73188835, 0.26811165])

In [20]:
np.stack(proba_global, axis=0)

array([[0.73188835, 0.26811165],
       [0.87269235, 0.12730765],
       [0.92160381, 0.07839619],
       ...,
       [0.96059179, 0.03940821],
       [0.96705772, 0.03294228],
       [0.98617029, 0.01382971]])

In [21]:
ss_test = ss.copy()

ss_test[['Stay','Attrition']] = proba_global

In [22]:
ss_test

,id,Attrition,Stay
0,1677,0.268112,0.731888
1,1678,0.127308,0.872692
2,1679,0.078396,0.921604
3,1680,0.061298,0.938702
4,1681,0.523901,0.476099
...,...,...,...
1114,2791,0.075159,0.924841
1115,2792,0.012388,0.987612
1116,2793,0.039408,0.960592
1117,2794,0.032942,0.967058


In [23]:
ss_test[['id','Attrition']].to_csv('submission.csv', index = False)